In [1]:
from bs4 import BeautifulSoup as bs
from pyparsing import line_end
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.common.exceptions import NoSuchElementException


from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
import time
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [33]:
#국산
url = 'http://www.encar.com/dc/dc_carsearchlist.do?carType=kor#!%7B%22action%22%3A%22%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22PriceAsc%22%2C%22page%22%3A{}%2C%22limit%22%3A%2250%22%2C%22searchKey%22%3A%22%22%2C%22loginCheck%22%3Afalse%7D'

In [60]:
#외제차
url = 'http://www.encar.com/fc/fc_carsearchlist.do?carType=for#!%7B%22action%22%3A%22(And.Hidden.N._.CarType.N.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22PriceAsc%22%2C%22page%22%3A{}%2C%22limit%22%3A%2250%22%2C%22searchKey%22%3A%22%22%2C%22loginCheck%22%3Afalse%7D'

In [66]:
#전기
url = "http://www.encar.com/ev/ev_carsearchlist.do?carType=ev#!%7B%22action%22%3A%22%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22PriceAsc%22%2C%22page%22%3A{}%2C%22limit%22%3A%2250%22%2C%22searchKey%22%3A%22%22%2C%22loginCheck%22%3Afalse%7D"

In [74]:
#화물
url = "http://www.encar.com/tr/tr_carsearchlist.do#!%7B%22action%22%3A%22%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22PriceAsc%22%2C%22page%22%3A{}%2C%22limit%22%3A%2250%22%2C%22searchKey%22%3A%22%22%2C%22loginCheck%22%3Afalse%7D"

In [75]:
ids = []
failed_pages = []

In [76]:
n =1

In [77]:
driver.get(url.format(n))
added = [0]*10
re_time = 0
while True:
    try:
        prev_id_numb = len(ids)
        time.sleep(1)
        nextbutt = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#pagination > span.next")))
        html = driver.page_source

        soup = bs(html, 'html.parser')
        trs = soup.find_all('tr', attrs={'data-impression': True, 'data-index': True})
        for tr in trs:
            tmp = tr['data-impression'].split('|')[0]
            ids.append(tmp)

        #중복 제거
        ids = list(set(ids))

        pres_id_numb = len(ids)
        d_numb = pres_id_numb - prev_id_numb
        added[n] += d_numb
        print('페이지 : ', n,'   id_in_page :', len(trs), '   추가된ids수 : ', added[n], "   ids : ", pres_id_numb , )

        #새로고침
        if added[n] < 40:
            if re_time < 5:
                driver.refresh()
                time.sleep(2)
                print("새로고침")
                re_time +=1
                continue
            else:
                re_time = 0

        n += 1
        pages = soup.find('span',class_='page')

        added.append(0)
        if pages:
            print("page있음")
            driver.get(url.format(n))
            continue
        nextbutt.click()

    except Exception as e:
        print('오류 : 페이지 : ', n, "ids : ", len(ids) )
        failed_pages.append(n)  # 오류가 발생한 페이지 번호 추가
        n += 1
        driver.get(url.format(n))
        print(e)
        continue


페이지 :  1    id_in_page : 59    추가된ids수 :  58    ids :  58
page있음
페이지 :  2    id_in_page : 59    추가된ids수 :  58    ids :  116
page있음
페이지 :  3    id_in_page : 59    추가된ids수 :  56    ids :  172
page있음
페이지 :  4    id_in_page : 59    추가된ids수 :  58    ids :  230
page있음
페이지 :  5    id_in_page : 59    추가된ids수 :  58    ids :  288
page있음
페이지 :  6    id_in_page : 59    추가된ids수 :  58    ids :  346
page있음
페이지 :  7    id_in_page : 59    추가된ids수 :  52    ids :  398
page있음
페이지 :  8    id_in_page : 59    추가된ids수 :  52    ids :  450
page있음
페이지 :  9    id_in_page : 59    추가된ids수 :  57    ids :  507
page있음
페이지 :  10    id_in_page : 59    추가된ids수 :  57    ids :  564
page있음
페이지 :  11    id_in_page : 59    추가된ids수 :  58    ids :  622
page있음
페이지 :  12    id_in_page : 59    추가된ids수 :  58    ids :  680
page있음
페이지 :  13    id_in_page : 59    추가된ids수 :  55    ids :  735
page있음
페이지 :  14    id_in_page : 59    추가된ids수 :  56    ids :  791
page있음
페이지 :  15    id_in_page : 59    추가된ids수 :  58    ids :  849
page있음
페이지 :

KeyboardInterrupt: 

In [78]:
len(ids)

10585

In [79]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()

cr_table_quary = """CREATE TABLE used_cars_id_ecar (
    id varchar(10) PRIMARY KEY not null
)"""

insert_query = "INSERT IGNORE INTO used_cars_id_ecar VALUES (%s)"

leni = len(ids)
j = 0
for id in ids:
    cur.execute(insert_query, (id))
    print('진행중 : ', j , '/', leni)
    j += 1
print('완료')
conn.commit()
conn.close()


진행중 :  0 / 10585
진행중 :  1 / 10585
진행중 :  2 / 10585
진행중 :  3 / 10585
진행중 :  4 / 10585
진행중 :  5 / 10585
진행중 :  6 / 10585
진행중 :  7 / 10585
진행중 :  8 / 10585
진행중 :  9 / 10585
진행중 :  10 / 10585
진행중 :  11 / 10585
진행중 :  12 / 10585
진행중 :  13 / 10585
진행중 :  14 / 10585
진행중 :  15 / 10585
진행중 :  16 / 10585
진행중 :  17 / 10585
진행중 :  18 / 10585
진행중 :  19 / 10585
진행중 :  20 / 10585
진행중 :  21 / 10585
진행중 :  22 / 10585
진행중 :  23 / 10585
진행중 :  24 / 10585
진행중 :  25 / 10585
진행중 :  26 / 10585
진행중 :  27 / 10585
진행중 :  28 / 10585
진행중 :  29 / 10585
진행중 :  30 / 10585
진행중 :  31 / 10585
진행중 :  32 / 10585
진행중 :  33 / 10585
진행중 :  34 / 10585
진행중 :  35 / 10585
진행중 :  36 / 10585
진행중 :  37 / 10585
진행중 :  38 / 10585
진행중 :  39 / 10585
진행중 :  40 / 10585
진행중 :  41 / 10585
진행중 :  42 / 10585
진행중 :  43 / 10585
진행중 :  44 / 10585
진행중 :  45 / 10585
진행중 :  46 / 10585
진행중 :  47 / 10585
진행중 :  48 / 10585
진행중 :  49 / 10585
진행중 :  50 / 10585
진행중 :  51 / 10585
진행중 :  52 / 10585
진행중 :  53 / 10585
진행중 :  54 / 10585
진행중 :  55 / 10585
진행